In [1]:
import openai 
import os 
import random 
random.seed(42)

def gauss_2d(mu, sigma=0.01):
    x = random.gauss(mu[0], sigma)
    y = random.gauss(mu[1], sigma)
    return (x, y)

openai.api_key = 'your key'  
models = openai.Model.list()
# for model in models['data']:
#     print(model['id'])
print("Done.")

Done.


### LLM-GROP 
LLM-GROP is a task planner based on large language model, such as ChatGPT.   

Specifically, they experiment a table setting task which is required to know the place position of the object.

There are two examples about it 


[Template 1: $\bold{Symbolic \ Spatial \ Reasoning}$] 

The goal is to set a dining table with objects. 

The symbolic spatial relationship between objects includes [spatial relationships].

[examples: a fork and a knife are placed on the left and right respectively]. 

What is a typical way of positioning [objects] on a table? 

[notes]




[Template 2: $\bold{Geometric \ Spatial\  Reasoning}$] 

[object A] is placed [spatial relationship] [object B]. 

How many centimeters [spatial relationship] [object B] should [object A] be placed?


First, this method gets a proper object position from LLM follwoing the above prompt.

Then, they use this position as a mean of a distribution to use it as a sample distribution.

### Symbolic Spatial Reasoning  
The goai is to place a dish in the kitchen. 


The symbolic spatial relationship betwen objects includes, a dish is suitable to be placed on a dish rack


What is a typical way of positioning a disth on a dish rack?

### Geometric Spatial Reasoning 
Other dishes are placed next to the red dish on the dish rack, 

How many centimeters red dish should other dishes be placed? 


In [4]:
engine = "text-davinci-003"
prompt = '''
The goal is to place a plate in the kitchen. 
In terms of the symbolic spatial relationship between objects, 
a plate is suitable for placement on a dish rack. 
The rack has slots aligned along the y-axis, 
with an interval of 0.06m between each slot. 
What is the typical method for positioning a plate on a dish rack?
'''
response = openai.Completion.create(engine='text-davinci-003',
                                   prompt=prompt,
                                   max_tokens = 512,
                                   temperature =1.0)
print("Answer: ", response["choices"][0]['text'])

Answer:  
Typically, dishes are placed in the dish rack by selecting the appropriate size slot and sliding the plate in. The plate should fit snugly in the space so that it will not easily slide out. Depending on the orientation of the rack and the size of the plate, it may be helpful to adjust the orientation of the plate so that it fits well in the space. It may also be necessary to adjust the spacing between slots according to the size of the plate.


In [7]:
engine = "text-davinci-003"
prompt = '''
Some plates are placed on a dish rack where It is placed on {x: 0.8, y:0.06, z:0.82}m, 
and the distance between the plates is 0.06m.
Where a plate should be placed considering other plates?
'''

response = openai.Completion.create(engine='text-davinci-003',
                                   prompt=prompt,
                                   max_tokens = 512,
                                   temperature =1.0)
print("Answer: ", response["choices"][0]['text'])

Answer:  
The plate should be placed at {x: 0.86, y:0.12, z:0.82}m to maintain the distance of 0.06m from the other plates.


### Gaussian Distribution

In [18]:
import numpy as np 
# Gaussian Normal Distribution
mean    = [0.8, 0.12]
cov     = [[0.001,0],[0, 0.001]]
samples = np.random.multivariate_normal(mean, cov, 1)
print("Samples: {}, Shape: {}".format(samples, samples.shape))

Samples: [[0.84921195 0.08353019]], Shape: (1, 2)


In [19]:
engine = "text-davinci-003"
prompt = '''
The goal is to place a plate in the kitchen. 
In terms of the symbolic spatial relationship between objects, 
a plate is suitable for placement on a dish rack. 
The rack has slots aligned along the y-axis, 
with an interval of 0.06m between each slot. 
What is the typical method for positioning a plate on a dish rack?
'''
response = openai.Completion.create(engine='text-davinci-003',
                                   prompt=prompt,
                                   max_tokens = 512,
                                   temperature =1.0)
print("Answer: ", response["choices"][0]['text'])

Answer:  
The typical method for positioning a plate on a dish rack is to slide the plate along the slots, paying careful attention to ensure that the plate is level and centered in the slot. Depending on the size of the plate, it is usually best practice to ensure that at least two edges of the plate are resting within the slot for stability. Some dish racks may also have adjustable slots which allow for fine-tuning the fit and placement of the plate.


### 1. Position Rand

In [27]:
engine = "text-davinci-003"
prompt = '''
The goal is to place a plate in the kitchen. 
In terms of the symbolic spatial relationship between objects, 
a plate is suitable for placement on a dish rack. 
The rack has slots aligned along the y-axis, 
with an interval of 0.06m between each slot. 

The plate size is {x:0.14, y:0.14, z:0.02}m. and the rotation is {x:90, y:0, z:0}degree.
Some plates are placed in a rack where the coordinate is {x: 0.8, y:0.1, z:0.82}m, 
Where a plate should be placed in a slot considering other plates?
'''

response = openai.Completion.create(engine='text-davinci-003',
                                   prompt=prompt,
                                   max_tokens = 512,
                                   temperature =1.0)
print("Answer: ", response["choices"][0]['text'])

Answer:  
First, we need to determine the size of the slots in the dish rack. 
Assuming the plate size is 0.14m x 0.14m, then the size of the slots in the dish rack should be 0.2m x 0.14m. 

Next, we need to determine the current coordinate of the plate in relation to the slots. 
The current coordinate of the plate is {x: 0.8, y:0.1, z:0.82}m, so it is placed at the 8th slot along the y-axis. 

In order to determine where a plate should be placed in a slot considering other plates, we need to calculate the spacing between the plates and the slots. 
The plate size is 0.14m x 0.14m, and the interval between each slot is 0.06m. 
Therefore, the spacing between the plates and the slots should be 0.04m. 

Finally, we can calculate the exact location of the plate in the slot. 
Assuming we start counting from the leftmost slot, the location of the plate would be {x: 0.8, y:0.14, z:0.82}m. 
This is the exact location where the plate should be placed in a slot considering other plates.


In [3]:
for i in range(10):
    res = gauss_2d([0.8, 0.14],0.03)
    print("{}. Result: {}".format(i+1,res))

1. Result: (0.7966605241529702, 0.1610595117529659)
2. Result: (0.7961723514865134, 0.0950793975697713)
3. Result: (0.8099695503220316, 0.13197987564508495)
4. Result: (0.7934912394756442, 0.14347654360102566)
5. Result: (0.8069689321072017, 0.1749067605979743)
6. Result: (0.8196990952039601, 0.14331521532331498)
7. Result: (0.7778503519296555, 0.1095601289753685)
8. Result: (0.8073902658563361, 0.17933242481612147)
9. Result: (0.8012497059171015, 0.13681030118687648)
10. Result: (0.8159532866120262, 0.09639364105973967)


### 2. Position Rand

In [28]:
engine = "text-davinci-003"
prompt = '''
The goal is to place a plate in the kitchen. 
In terms of the symbolic spatial relationship between objects, 
a plate is suitable for placement on a dish rack. 
The rack has slots aligned along the y-axis, 
with an interval of 0.06m between each slot. 

The plate size is {x:0.14, y:0.14, z:0.02}m. and the rotation is {x:90, y:0, z:0}degree.
Some plates are placed in a rack where the coordinate is {x: 0.8, y:0., z:0.82}m, 
Where a plate should be placed in a slot considering other plates?
'''

response = openai.Completion.create(engine='text-davinci-003',
                                   prompt=prompt,
                                   max_tokens = 512,
                                   temperature =1.0)
print("Answer: ", response["choices"][0]['text'])

Answer:  
Assuming the other plates are placed with their y-axis aligned, 
the coordinates for the plate should be {x: 0.8, y: 0.06, z: 0.82}m.


### 3. Position Rand

In [29]:
engine = "text-davinci-003"
prompt = '''
The goal is to place a plate in the kitchen. 
In terms of the symbolic spatial relationship between objects, 
a plate is suitable for placement on a dish rack. 
The rack has slots aligned along the y-axis, 
with an interval of 0.06m between each slot. 

The plate size is {x:0.14, y:0.14, z:0.02}m. and the rotation is {x:90, y:0, z:0}degree.
Some plates are placed in a rack where the coordinate is {x: 0.76, y:0.06, z:0.82}m, 
Where a plate should be placed in a slot considering other plates?
'''

response = openai.Completion.create(engine='text-davinci-003',
                                   prompt=prompt,
                                   max_tokens = 512,
                                   temperature =1.0)
print("Answer: ", response["choices"][0]['text'])

Answer:  
Assuming that the plates all have the same size and orientation, the plate should be placed in the slot directly above or below the existing plate. This would have the same x-coordinate (0.76 m) and the same z-coordinate (0.82 m), but the y-coordinate would be either 0.00 m or 0.12 m, respectively.


### 4. Position Rand

In [30]:
engine = "text-davinci-003"
prompt = '''
The goal is to place a plate in the kitchen. 
In terms of the symbolic spatial relationship between objects, 
a plate is suitable for placement on a dish rack. 
The rack has slots aligned along the y-axis, 
with an interval of 0.06m between each slot. 

The plate size is {x:0.14, y:0.14, z:0.02}m. and the rotation is {x:90, y:0, z:0}degree.
Some plates are placed in a rack where the coordinate is {x: 0.76, y:-0.1, z:0.82}m, 
Where a plate should be placed in a slot considering other plates?
'''

response = openai.Completion.create(engine='text-davinci-003',
                                   prompt=prompt,
                                   max_tokens = 512,
                                   temperature =1.0)
print("Answer: ", response["choices"][0]['text'])

Answer:  
Considering the size and orientation of the plate, it should be placed in the slot at {x: 0.76, y: -0.04, z: 0.82}m (the 4th slot down from the top). This slot should be able to accommodate the plate, and will fit in between the other plates that are already placed.


### 5. Position Rand

In [33]:
engine = "text-davinci-003"
prompt = '''
The goal is to place a plate in the kitchen. 
In terms of the symbolic spatial relationship between objects, 
a plate is suitable for placement on a dish rack. 
The rack has slots aligned along the y-axis, 
with an interval of 0.06m between each slot. 

The plate size is {x:0.14, y:0.14, z:0.02}m. and the rotation is {x:90, y:0, z:0}degree.
Some plates are placed in a rack where the coordinate is {x: 0.85, y:0.06, z:0.82}m, 
Where a plate should be placed in a slot considering other plates?
'''

response = openai.Completion.create(engine='text-davinci-003',
                                   prompt=prompt,
                                   max_tokens = 512,
                                   temperature =1.0)
print("Answer: ", response["choices"][0]['text'])

Answer:  
Assuming that the plate is being placed in the dish rack in an upright position, the ideal coordinate for placement should be x: 0.85, y: 0.12, z: 0.82 m. This is because this coordinate places the plate in a slot which aligns with the other plates in the dish rack.


### 6. Position Rand

In [4]:
engine = "text-davinci-003"
prompt = '''
The goal is to place a plate in the kitchen. 
In terms of the symbolic spatial relationship between objects, 
a plate is suitable for placement on a dish rack. 
The rack has slots aligned along the y-axis, 
with an interval of 0.06m between each slot. 

The plate size is {x:0.14, y:0.14, z:0.02}m. and the rotation is {x:90, y:0, z:0}degree.
Some plates are placed in a rack where the coordinate is {x: 0.8, y:0.14, z:0.82}m, 
Where a plate should be placed in a slot considering other plates?
'''

response = openai.Completion.create(engine='text-davinci-003',
                                   prompt=prompt,
                                   max_tokens = 512,
                                   temperature =1.0)
print("Answer: ", response["choices"][0]['text'])

Answer:  
The plate should be placed in a slot at a coordinate of {x: 0.8, y:0.14 + 0.06, z:0.82}. This coordinate accounts for the sizes of the plates already in the rack and also ensures the plate is properly spaced and aligned.


### 7. Position Rand

In [5]:
engine = "text-davinci-003"
prompt = '''
The goal is to place a plate in the kitchen. 
In terms of the symbolic spatial relationship between objects, 
a plate is suitable for placement on a dish rack. 
The rack has slots aligned along the y-axis, 
with an interval of 0.06m between each slot. 

The plate size is {x:0.14, y:0.14, z:0.02}m. and the rotation is {x:90, y:0, z:0}degree.
Some plates are placed in a rack where the coordinate is {x: 0.8, y:-0.14, z:0.82}m, 
Where a plate should be placed in a slot considering other plates?
'''

response = openai.Completion.create(engine='text-davinci-003',
                                   prompt=prompt,
                                   max_tokens = 512,
                                   temperature =1.0)
print("Answer: ", response["choices"][0]['text'])

Answer:  
Assuming that the other plates are already occupying the slots, 
the plate should be placed in the slot with the x-coordinate of 0.8 and y-coordinate of -0.06 (which is 0.06 m less than the y-coordinate of the other plates).  Thus, the plate should be placed in the slot at {x: 0.8, y: -0.06, z: 0.82} m.


### 8. Position Rand

In [6]:
engine = "text-davinci-003"
prompt = '''
The goal is to place a plate in the kitchen. 
In terms of the symbolic spatial relationship between objects, 
a plate is suitable for placement on a dish rack. 
The rack has slots aligned along the y-axis, 
with an interval of 0.06m between each slot. 

The plate size is {x:0.14, y:0.14, z:0.02}m. and the rotation is {x:90, y:0, z:0}degree.
Some plates are placed in a rack where the coordinate is {x: 0.85, y:-0.04, z:0.82}m, 
Where a plate should be placed in a slot considering other plates?
'''

response = openai.Completion.create(engine='text-davinci-003',
                                   prompt=prompt,
                                   max_tokens = 512,
                                   temperature =1.0)
print("Answer: ", response["choices"][0]['text'])

Answer:  
Considering the size and rotation of the plate, the slot to place the plate should be aligned with the y-axis and located at a coordinate of {x: 0.85, y:-0.02, z:0.82}m. This coordinates takes into account the other plates already in the rack and allows the plate to fit and rotate properly in the slot.


### 9. Position Rand

In [8]:
engine = "text-davinci-003"
prompt = '''
The goal is to place a plate in the kitchen. 
In terms of the symbolic spatial relationship between objects, 
a plate is suitable for placement on a dish rack. 
The rack has slots aligned along the y-axis, 
with an interval of 0.06m between each slot. 

The plate size is {x:0.14, y:0.14, z:0.02}m. and the rotation is {x:90, y:0, z:0}degree.
Some plates are placed in a rack where the coordinate is {x: 0.8, y:0.05, z:0.82}m, 
Where a plate should be placed in a slot considering other plates?
'''

response = openai.Completion.create(engine='text-davinci-003',
                                   prompt=prompt,
                                   max_tokens = 512,
                                   temperature =1.0)
print("Answer: ", response["choices"][0]['text'])

Answer:  
Assuming the slot is at the center of the rack, the plate should be placed in the slot at the coordinate {x:0.8, y:0.11, z:0.82}m (half of the slot interval of 0.06m beyond the slot at {x:0.8, y:0.05, z:0.82}m). 
The plate should be oriented so that the x-axis of the plate is aligned with the y-axis of the rack, and the rotation values should be {x:90, y:0, z:0}degrees.


### 10. Position Rand

In [10]:
engine = "text-davinci-003"
prompt = '''
The goal is to place a plate in the kitchen. 
In terms of the symbolic spatial relationship between objects, 
a plate is suitable for placement on a dish rack. 
The rack has slots aligned along the y-axis, 
with an interval of 0.06m between each slot. 

The plate size is {x:0.14, y:0.14, z:0.02}m. and the rotation is {x:90, y:0, z:0}degree.
Some plates are placed in a rack where the coordinate is {x: 0.82, y:0.13, z:0.82}m, 
Where a plate should be placed in a slot considering other plates?
'''

response = openai.Completion.create(engine='text-davinci-003',
                                   prompt=prompt,
                                   max_tokens = 512,
                                   temperature =1.0)
print("Answer: ", response["choices"][0]['text'])

Answer:  
Assuming the dish rack has four slots, to place the plate in the slot considering other plates, the plate should be placed in the slot with the following coordinates: 
{x: 0.82, y: 0.19, z: 0.82}m. The reason is that the plate should be placed in the slot with the highest y-coordinate that is not already occupied. Each slot interval is 0.06m and so the y-coordinate is increased by 0.06m.


In [8]:
# Without slot interval

engine = "text-davinci-003"
prompt = '''
The goal is to place a plate in the kitchen. 
In terms of the symbolic spatial relationship between objects, 
a plate is suitable for placement on a dish rack. 
The rack has slots aligned along the y-axis

The plate size is {x:0.14, y:0.02, z:0.14}m.
Some plates are placed in a rack where the coordinate is {x: 0.82, y:0.13, z:0.82}m, 
How many centimeters a plate should other plates be placed?
'''

response = openai.Completion.create(engine='text-davinci-003',
                                   prompt=prompt,
                                   max_tokens = 512,
                                   temperature =1.0)
print("Answer: ", response["choices"][0]['text'])

Answer:  
The distance between plates should be at least 0.02 meters (2 cm) for safe storage.
